# BigQuery ARIMA PLUS Forecasting

Author: Marco Pellegrino
Year: 2024

**Description**\
The following SQL queries train an ARIMA+ model on BigQuery to forecast the last 30 days of a page time series.

### Split Time Series in Training and Test set

In [ ]:
-- Create a temporary table to store the last 30 dates
CREATE OR REPLACE TABLE web-traffic-time-series.web_traffic_dataset.temp_last_30_dates AS (
  SELECT DISTINCT date
  FROM `web-traffic-time-series.web_traffic_dataset.web_traffic_tb`
  ORDER BY date DESC
  LIMIT 30
);

-- Create the training set
CREATE OR REPLACE TABLE web-traffic-time-series.web_traffic_dataset.web_traffic_training_tb AS (
  SELECT
    date,
    page,
    traffic
  FROM `web-traffic-time-series.web_traffic_dataset.web_traffic_tb`
  WHERE
    AND date NOT IN (SELECT date FROM web-traffic-time-series.web_traffic_dataset.temp_last_30_dates)
);

-- Create the test set
CREATE OR REPLACE TABLE web-traffic-time-series.web_traffic_dataset.web_traffic_test_tb AS (
  SELECT
    date,
    page,
    traffic
  FROM `web-traffic-time-series.web_traffic_dataset.web_traffic_tb`
  WHERE
    AND date IN (SELECT date FROM web-traffic-time-series.web_traffic_dataset.temp_last_30_dates)
);

-- Drop the temporary table if no longer needed
DROP TABLE web-traffic-time-series.web_traffic_dataset.temp_last_30_dates;

### Train ARIMA PLUS Model on Training set

In [ ]:
CREATE OR REPLACE MODEL `web-traffic-time-series.web_traffic_dataset.web_traffic_arima_model`
OPTIONS
  (model_type = 'ARIMA_PLUS',
  time_series_timestamp_col = 'date',
  time_series_data_col = 'traffic',
  auto_arima = TRUE, -- auto hyperparameter tuning of p, d, q
  horizon = 30
  ) AS (SELECT
          date,
          traffic
        FROM `web-traffic-time-series.web_traffic_dataset.web_traffic_training_tb`
        )

### Evaluate ARIMA PLUS Model on Test set

In [ ]:
SELECT
  *
FROM
  ML.EVALUATE(MODEL `web_traffic_dataset.web_traffic_arima_model`,
    (
    SELECT
      CAST(date AS TIMESTAMP) date,
      traffic
    FROM
      `web_traffic_dataset.web_traffic_test_tb`))

Results:
* MAE: 11.17
* MSE: 308.23
* RMSE: 17.55

### Forecast last month of data to be plotted with Looker Studio

In [ ]:
WITH forecast_traffic_tb AS (
      SELECT
          DATE(forecast_timestamp) AS day,
				  CAST(forecast_value AS INT64) AS forecast_traffic, 
					CAST(prediction_interval_lower_bound AS INT64) AS forecast_traffic_lower_bound,
				  CAST(prediction_interval_upper_bound AS INT64) AS forecast_traffic_upper_bound
        FROM 
          ML.FORECAST(MODEL web_traffic_dataset.web_traffic_arima_model, STRUCT(30 AS horizon, 0.9 AS confidence_level)) ),
                 
    real_traffic_tb AS (
        SELECT date, traffic as real_traffic
        FROM web_traffic_dataset.web_traffic_training_tb
        UNION ALL
        SELECT date, traffic as real_traffic
        FROM web_traffic_dataset.web_traffic_test_tb
        ORDER BY date DESC) 
SELECT date, real_traffic, forecast_traffic, forecast_traffic_lower_bound, forecast_traffic_upper_bound
FROM real_traffic_tb 
LEFT JOIN forecast_traffic_tb 
ON real_traffic_tb.date = forecast_traffic_tb.day
ORDER BY date DESC;

![Plot language analysis](../plots_looker/plot_arima_plus_forecast.png)